<a href="https://colab.research.google.com/github/nyac-1/trading-financial-announcements/blob/main/new_sent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

from google.colab import drive; drive.mount('/content/gdrive', force_remount=True)
%cd '/content/gdrive/My Drive/Colab Notebooks'

import pandas as pd
import glob

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks


In [ ]:
!pip install pdflib


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
dir='/content/gdrive/My Drive/nlp'
pdf_dir = '/aem'

pdfs = glob.glob(dir+pdf_dir+'/*.pdf')
df = pd.read_csv(dir+"/text.csv")

In [ ]:
dates = []
names = []

for i,pdf in enumerate(pdfs):
    dates.append(pdf.replace(dir+pdf_dir+"/","").split()[0])
    names.append(pdfs[i].split("/")[-1])


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pdflib import Document

from transformers import BertTokenizer, BertForSequenceClassification
import torch
from transformers import pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

In [ ]:
label_list=['neutral','positive','negative']

In [ ]:
sentLs=[]
for i,pdf in enumerate(pdfs):
    doc = Document(pdf)
    sentSum=0
    for page in doc:
        txt=' \n'.join(page.lines).strip()
        inputs = tokenizer(txt, return_tensors="pt",max_length=512,truncation=True)
        outputs = finbert(**inputs)
        sent=torch.argmax(outputs[0])
        if sent==1: sentSum+=1
        elif sent==2: sentSum-=1
    
    sentLs.append(sentSum)
    print(i)

461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546


In [ ]:
new_df = pd.DataFrame()
new_df['sent']=sentLs
new_df['name']=names
new_df.to_csv("sent-ana.csv", index = False)

In [ ]:
import pickle
with open('sent.pkl', 'wb') as f:
    pickle.dump(sentLs, f)
with open('names.pkl', 'wb') as fs:
    pickle.dump(names, fs)